# Time Series Analysis with Great Britain Power Consumption Data

[Enrique Z. Losoya](https://orcid.org/0000-0001-7763-3349) and [Jian Tao](https://orcid.org/0000-0003-4228-6089), Texas A&M University.

Updated: Jan. 2, 2023.

The data set is downloaded from [Open Power System Data (last updated on 10/06/2020)](https://data.open-power-system-data.org/time_series/2020-10-06)
A local copy of the data can be found in time_series_30min_singleindex.csv. The descrition of the data set can be found in time_series_30min_singleindex.txt.

## 1. Read and Clean Data

In [ ]:
# import pandas and make sure the plots show properly in Jupyter notebook.
data_url="https://data.open-power-system-data.org/time_series/2020-10-06/time_series_30min_singleindex.csv"
data_file="time_series_30min_singleindex.csv"
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
if not os.path.exists(data_file):
    !wget $data_url

In [ ]:
# read the data
df=pd.read_csv(data_file)
# let's take a look at the data right after it is imported.
df
df.info()

In [ ]:
# we only want to look into the total energy consumption, solar energy generation, and wind energy generation
df=df[["utc_timestamp","GB_GBN_load_actual_entsoe_transparency", "GB_GBN_solar_generation_actual", "GB_GBN_wind_generation_actual"]]

# rename the column names to make it easy to use them
df.columns=["time", "total", "solar", "wind"]

# let's see how df looks like now. 
df

In [ ]:
# make the time as the index of the rows for easy manipulation with pandas builtin functions.
df["time"] = pd.to_datetime(df["time"])
df.set_index(["time"], inplace=True)

# let's take a look at df that we will be working on. 
# Don't worry about NaNs as there are always missing data points in real world datasets.
df

In [ ]:
df.index

In [ ]:
# some more information about the data set
df.info()

In [ ]:
# statistics of the data set. Here T is to transpose the matrix.
df.describe().T

In [ ]:
# It is not a good practice to replace missing time series data with medians or means.
# Instead, it is usally a better choice to use forward filling, backward 
# filling, linear interpolation, mean of nearest neighbours, etc.
df_filled=df.bfill().ffill()

In [ ]:
df_filled.info()
df_filled

In [ ]:
# we can save the file in a CSV file that could be directly used next time.
df.to_csv("GB_power.csv")

## 2. Aggregration - aggregate data over a certain time period

In [ ]:
# let's try to resample the data every 5 days and output the mean value.

df_5days_mean = df_filled[["total","solar","wind"]].resample("5d").mean(); df_5days_mean

In [ ]:
sns.set(rc={'figure.figsize':(12,4)})
df_5days_mean.plot(title="Great Britain Power Consumption (MWh)");

In [ ]:
# aggregate the data to get the weekly max

df_weekly_max = df_filled[["total","solar","wind"]].resample("w").max(); df_weekly_max

In [ ]:
df_filled["total"].plot()

In [ ]:
df_weekly_max.plot(title="Great Britain Power Consumption (MWh)");

In [ ]:
# let's try to resample monthly and output the monthly median

df_monthly_median = df_filled[["total","solar","wind"]].resample("m").median(); df_monthly_median

In [ ]:
df_monthly_median.plot(title="Great Britain Power Consumption (MWh)");

In [ ]:
df_filled[["total","solar","wind"]].resample("d").median().plot(title="Great Britain Power Consumption (MWh)").figure.savefig("energy.png")

## 3. Rolling Windows

In [ ]:
# center = True below means the rolling mean would be calculated and placed next 
# to the center of the bin (with a width of 7 days). As a result, the first 3 and last
# 3 rows are NaNs.

df_7lags_rol = df_filled.rolling(window = 7, center = True).mean(); df_7lags_rol

In [ ]:
df_7lags_rol.plot(title="Great Britain Power Consumption (MWh)", figsize=(15,4));

In [ ]:
df_filled["2018"].resample("H").median().plot()

In [ ]:
df_monthly_median["total"].plot(title="Total Great Britain Power Consumption (MWh)", figsize=(15,4)).figure.savefig("energy.png");

## Detrend with Differencing

In [ ]:
# Pandas has a builtin function for detrending. We can easily get the differencing with different orders.
df_1st_diff=df_filled.resample("d").median().diff(periods=1)

In [ ]:
df_1st_diff.plot()

In [ ]:
df_1st_diff = df_1st_diff[2:]

In [ ]:
df_1st_diff["total"].plot(title="1st Order Differencing of the Total Power Usage").figure.savefig("energy.png");

In [ ]:
from statsmodels.tsa.stattools import kpss

def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

In [ ]:
kpss_test(df_1st_diff["total"])

In [ ]:
# ADF test for checking the stationarity of a time series.
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
adf_test(df_1st_diff["total"])

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
sns.set(rc={'figure.figsize':(12,4)})
plot_acf(df_1st_diff["total"]);
plot_pacf(df_1st_diff["total"]);

In [ ]:
pd.plotting.autocorrelation_plot(df_1st_diff["total"])

In [ ]:
df_filled['Month'] = df_filled.index.month

In [ ]:
df_filled

In [ ]:
# this is just for visualization purpose. 
import matplotlib.pyplot as plt
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['total', 'solar', 'wind'], axes):
    sns.boxplot(data=df_filled, x='Month', y=name, ax=ax)
    ax.set_ylabel('MWh')
    ax.set_title(name)
    # Keep the x-axis label for only the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')
plt.savefig("energy.png")        

## Import Modules and Make Time Series Stationary

In [ ]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

ARIMA is only working for stationary time series, so we need to detrend the data first.

In [ ]:
# We must skip the first NaN due to the differencing. If we don't, the NaN will contaminate the rest of the calculations.
X = df_filled["total"].resample("h").mean().diff(periods=1).values[2:]

In [ ]:
plt.plot(X)

In [ ]:
#X_train, X_test = train_test_split(X, test_size=0.20, random_state=42)
X_train = X[:40319]
X_test = X[40319:]

In [ ]:
X.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## AutoRegressive Model

In [ ]:
model_ar = AR(X_train)

In [ ]:
model_ar_fit = model_ar.fit()

In [ ]:
ar_predict = model_ar_fit.predict(start=40319, end=50399)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(ar_predict)
plt.plot(X_test, c="r")

It seems there are some outliers at the very beginning of the time series. We might want to remove them.

## ARIMA Model

In [ ]:
# ARIMA needs three parameters: p, d, q.
# p = periods taken for autoregressive model
# d = integrated order, difference
# q = periods in moving average model
model_arima = ARIMA(X_train, order=(2,1,0))

In [ ]:
model_arima_fit=model_arima.fit()
print(model_arima_fit.aic)

In [ ]:
n_pred = 100 # Lead time, or forecasting horizon -- the number of steps ahead for out of sample forecast 
pred_array, se_array, CI_array = model_arima_fit.forecast(steps=n_pred,alpha=0.03) # alpha: confidence level
plt.figure(figsize=(12,4))
pred_array_index = range(40319, 40419)
plt.plot(pred_array_index, X_test[:100])
plt.plot(pred_array_index, pred_array, color = "red")
plt.fill_between(pred_array_index, CI_array[:,0], CI_array[:,1], color = "k", alpha = .03 )
plt.title('Forecast of the simulated data')
plt.show()

## Find the Optimal Configuration for an ARIMA Model

In [ ]:
# data  : the train data in forms of numpy array
# order : the maximum order of p, d, q for the grid search
# return: optimal (p, d, q)

# example: myorder = arima_order(X_train)

def arima_order(data, order=10, verbose=True):
  import warnings
  import itertools
  p=d=q=range(0,order)
  pdq = list(itertools.product(p,d,q))
  warnings.filterwarnings("ignore")
  aic_pair={}
  min_aic=9999999
  min_order=(0,0,0)
  for o in pdq:
    try:
      model_arima = ARIMA(data, order=o)
      model_arima_fit=model_arima.fit()
      fit_aic = model_arima_fit.aic
      if verbose: print(o, fit_aic)
      if not np.isnan(fit_aic):
        if min_aic>fit_aic:
          min_aic = fit_aic
          min_order=o
      aic_pair.update({o, rmodel_arima_fit.aic})
    except:
      continue
  return min_order

In [ ]:
arima_order(X, order=4)

In [ ]:
model_arima = ARIMA(X_train, order=(2, 0, 2))
model_arima_fit=model_arima.fit()

## Make Predictions with ARIMA Model

### Prediction for the stationary component

In [ ]:
n_pred = 100 # Lead time, or forecasting horizon -- the number of steps ahead for out of sample forecast 
pred_array, se_array, CI_array = model_arima_fit.forecast(steps=n_pred,alpha=0.03) # alpha: confidence level
plt.figure(figsize=(12,4))
pred_array_index = range(40319, 40419)
plt.plot(pred_array_index, X_test[:100])
plt.plot(pred_array_index, pred_array, color = "red")
plt.fill_between(pred_array_index, CI_array[:,0], CI_array[:,1], color = "k", alpha = .03 )
plt.title('Forecast of the simulated data')
plt.show()

### Inverse Difference
Remember that we did differencing once, so we will need to reverse that operation. The following function is to reverse a differenced time series.

In [ ]:
def inverse_difference(h, x, interval=1):
	return x + h[-interval]

### Make predictions

In [ ]:
history = [x for x in X]
day =1
forecast = model_arima_fit.forecast(steps=30)[0]
for y in forecast:
	inverted = inverse_difference(history, y)
	print('Day %d: %6.2f' % (day, inverted))
	history.append(inverted)
	day += 1